In [14]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_functions
%run My_functions.py
%autoreload 1
%matplotlib inline
warnings.simplefilter(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%reset

In [15]:
client = Client()

In [16]:
# Data Import
hourly_raw_data = dd.read_csv("hour.csv", blocksize=16e4)

In [17]:
### Changing the date type
hourly_raw_data["dteday"].map_partitions(
    pd.to_datetime, format="%Y-%m-%d", meta=("datetime64[ns]")
)

Dask Series Structure:
npartitions=8
    datetime64[ns]
               ...
         ...      
               ...
               ...
dtype: datetime64[ns]
Dask Name: to_datetime, 40 tasks

In [18]:
# def isDayLightDask(df):
#     df.apply(lambda x: isDaylight(x), axis=1)
#     return df

In [19]:
# hourly_raw_data['isDaylight']=0
# hourly_raw_data['isNoon']=0

# hourly_raw_data.map_partitions(dask_it,isDaylight )

### Adding the temp atemp windspeed hum relative to the last 7 days value


In [20]:
to_relative  = ['temp', 'atemp', 'hum','windspeed']
hourly_raw_data = relative_values(hourly_raw_data, to_relative)

### Adding `RushHour-High` &    `RushHour-Med`  &   `RushHour-Low`


In [21]:
hourly_raw_data['RushHour-High'] = 0
hourly_raw_data['RushHour-Med'] = 0
hourly_raw_data['RushHour-Low'] = 0
hourly_raw_data = hourly_raw_data.map_partitions(lambda df: df.apply(lambda x: addRushHourFlags(x), axis=1))

## Splitting Data

In [22]:
workingdays = num_name(hourly_raw_data.loc[(hourly_raw_data['workingday'].isin([1]) )])
holidays = num_name(hourly_raw_data.loc[(~hourly_raw_data['workingday'].isin([1]) )])

## Mean of the past 3 weeks during the same hour

In [ ]:
holidays['mean_per_hour'] = 0
workingdays['mean_per_hour'] = 0   
holidays = holidays.map_partitions(lambda df: df.apply(lambda x: mean_per_hour_3weeks_row(x , holidays), axis=1))
workingdays = workingdays.map_partitions(lambda df: df.apply(lambda x: mean_per_hour_3weeks_row(x , holidays), axis=1))

### One hot Encoding | 2x for splitted datasets
For `season`, `weathersit`, `mnth`,`weekday`,`hr`

In [ ]:
category  = ['season', 'weathersit', 'mnth','weekday','hr']

workingdays = onehot_encode(workingdays,category)

holidays = onehot_encode(holidays,category)


## Genetic Programming | 2x for splitted datasets

In [ ]:
dates = workingdays['dteday']
registered = workingdays['registered']
casual = workingdays['casual']
instant = workingdays['instant']
workingdays = Genetic_P(workingdays.drop(['registered','casual','dteday' , 'instant'],axis=1),'cnt')
workingdays['instant'] = instant 
workingdays['dteday'] = dates
workingdays['registered'] = registered
workingdays['casual'] = casual

In [ ]:
instant= holidays['instant']
dates = holidays['dteday']
registered = holidays['registered']
casual = holidays['casual']
holidays = Genetic_P(holidays.drop(['registered','casual','dteday','instant'],axis=1),'cnt')
holidays['instant'] = instant
holidays['dteday'] = dates
holidays['registered'] = registered
holidays['casual'] = casual

In [ ]:
w = workingdays.to_csv("workingdays_data_prepared.csv", index=False,compute=False)
h = holidays.to_csv("weekends_holi_data_prepared.csv", index=False,compute=False)
_, l = dask.compute(w, h, workers= 7)

In [13]:
hourly_raw_data.compute(workers = 7)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,...,casual,registered,cnt,relative_temp,relative_atemp,relative_hum,relative_windspeed,RushHour-High,RushHour-Med,RushHour-Low
6,7,2011-01-01,1,0,1,6,0,6,0,1,...,2,0,2,-1.290994,-0.410671,0.792594,-0.408248,0,0,1
7,8,2011-01-01,1,0,1,7,0,6,0,1,...,1,2,3,-2.939874,-1.697564,2.828427,-0.377964,0,0,1
8,9,2011-01-01,1,0,1,8,0,6,0,1,...,1,7,8,0.840027,1.049718,-1.037749,-0.353553,0,1,0
9,10,2011-01-01,1,0,1,9,0,6,0,1,...,8,6,14,6.270697,5.732372,-0.664784,-0.333333,0,1,0
10,11,2011-01-01,1,0,1,10,0,6,0,1,...,12,24,36,4.451044,4.284212,-0.619389,8.637677,1,0,0
11,12,2011-01-01,1,0,1,11,0,6,0,1,...,26,30,56,2.080626,0.964934,0.810243,3.213730,1,0,0
12,13,2011-01-01,1,0,1,12,0,6,0,1,...,29,55,84,2.708013,3.103062,-0.378272,2.214437,1,0,0
13,14,2011-01-01,1,0,1,13,0,6,0,2,...,47,47,94,2.610559,2.797839,-1.835309,1.922405,1,0,0
14,15,2011-01-01,1,0,1,14,0,6,0,2,...,35,71,106,2.041551,2.133943,-1.579824,1.523345,1,0,0
15,16,2011-01-01,1,0,1,15,0,6,0,2,...,40,70,110,1.521491,1.583008,-0.104379,1.476610,1,0,0
